<img align="center" src="https://sun9-54.userapi.com/c543107/v543107547/23b15/U55ygyGaHYc.jpg" width="550">

Соавторы - Лебедева Виктория, Шарапова Екатерина, Короваева Анастасия.

### Наша задача:

Для нашего проекта мы решили поставить задачу, которая будет каким-то образом связана с направлением нашего обучения в Высшей школе экономики. Так у нас возникла идея, что в рекламных целях нам надо создать собственный паблик, темой которого будет искусство фотографии и их последующая обработка.

Для анализа мы взяли 7 пабликов, связанные с фотографией и фотошопом, в которых мы выделяем тематики для того, чтобы определить, какой контент будет в нашем паблике. Также мы качаем соцдем по этим 7 группам, чтобы определить примерную целевую аудиторию нашего будущего паблика.

### Качаем данные о любителях фотографии))

Для данной работы мы взяли 7 групп связанных с обработкой фотографий, в которых мы проанализировали их подписчиков. 

* [Обработка фото - Photoshop - Lightroom](https://vk.com/artmilart)
* [Фотография для души](https://vk.com/foto_forsoul)
* [Фотошоп-мастер](https://vk.com/phmaste)
* [Art Family](https://vk.com/club_of_photoshop)
* [Lightroom&Photoshop](https://vk.com/lr_school)
* [Журнал PhotoCasa](https://vk.com/journalphoto)
* [Photoshop + Творчество](https://vk.com/psclub)


In [9]:
import requests # библиотека для походов в интернет
import time     # пакет для работы с временем

In [84]:
# мой номер странички
myid = '216465800'# вставить номер странички

# версия используемого API
version = '5.103' 

# подгружаем токен
#with open('secret_token.txt') as f:
token =''

In [85]:
def vk_download(method, parameters):
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + str(token) + '&v=' + version
    response = requests.get(url) 
    infa = response.json()
    return infa 

In [86]:
method = 'users.get'
parameters = 'user_ids=' + myid

vk_download(method, parameters)

{'response': [{'id': 216465800,
   'first_name': 'Анастасия',
   'last_name': 'Короваева',
   'is_closed': False,
   'can_access_closed': True}]}

Ура, работает!

Берем 7 групп связанных с обработкой фотографий, в которых мы будем анализировать их подписчиков. 


In [23]:
publics = {
    'foto_forsoul': '16685',
    'artmilart': '4511982',
    'phmaste': '41454884',
    'club_of_photoshop': '7560633',
    'lr_school': '68024674',
    'journalphoto': '47332952',
    'psclub':'337'
      }

In [19]:
cur_id = publics['psclub']

Давайте узнаем сколько в группе человек

In [20]:
# узнаем сколько в группе человек
cur_infa = vk_download('groups.getMembers', 'group_id=' + str(cur_id))
cur_infa['response']['count']

549616

In [21]:
cur_infa['response']['count'] - 50000

499616

In [22]:
start_offset = max(0, cur_infa['response']['count'] - 50000)
start_offset

499616

In [24]:
def group_members_download(cur_id):
    
    # Сколько всего в группе людей
    cur_infa = vk_download('groups.getMembers', 'group_id=' + str(cur_id))
    
    # стартовая точка
    start_offset = max(0, cur_infa['response']['count'] - 5000)

    infa = [ ] # сюда буду сохранять людей

    # качаем по 1000 штук по циклу :) 
    for offset in range(start_offset, start_offset + 5000, 1000):

        time.sleep(0.4) # вк согласен работать 3 раза в секунду, 
                        # эта строка нужна, чтобы он не ругался 

        cur_infa = vk_download('groups.getMembers', 
                    'group_id=' + str(cur_id) + '&offset=' + str(offset))

        infa.extend(cur_infa['response']['items'])
    return infa

In [111]:
members.keys()

dict_keys([])

In [112]:
members = {} # сюда будем класть всех членов групп по очереди

for k,v in publics.items():
    print('Качаю группу', k)
    infa = group_members_download(v)
    members[k] = infa  

Качаю группу foto_forsoul
Качаю группу artmilart
Качаю группу phmaste
Качаю группу club_of_photoshop
Качаю группу lr_school
Качаю группу journalphoto
Качаю группу psclub


In [29]:
import pickle  # пакет для сохранения и подгрузки питонячих объектов

# сохраняем всё в файл group_members.pickle
#with open('group_members_new.pickle', 'wb') as f:
    #pickle.dump(members, f)
    
# Подгрузить данные из файлика можно вот так: 
with open('group_members.pickle', 'rb') as f:
    members = pickle.load(f)

#### Будем чистить участников групп и грузить соц-дем

Так как не все участники группы активны, нам необходимо почистить участников. Удалим тех, кто долго не появлялся в сети или удалил страницу. Заодно будем грузить соц-дем о участниках (день рождения. город, родной город и пол).

In [30]:
# Пакет time позволяет посмотреть сколько секунд прошло к текущему моменту. 
time.time()

1576788035.65495

In [32]:
fields = 'last_seen, bdate, city, home_town, sex'

In [33]:
def filter_members(memes_id):
    
    # сделали id текстом
    memes_id_text = [str(item) for item in memes_id]
    
    clean_memes_id = [ ] # список для хорших id 
    
    # качаем информацию кусочками по 300 штук
    for i in range(len(memes_id_text)//300):
                
        time.sleep(0.4) # немного подождали, чтобы вк нас не банил
        
                                                         # взяли текущие 300 человек из списка
        infa = vk_download('users.get','user_ids=' + ','.join(memes_id_text[i*300:(i+1)*300]) + '&fields={}'.format(fields))
 
        

        # чистим пользователей
        for item in infa['response']:

            try: # попробуй сделать вот это

                # если пользователь не удален, не приватный, а ещё недавно был в сети 
                if ('deactivated' not in item) and \
                   (item['is_closed'] == False) and \
                   (item['last_seen']['time'] >= time.time() - 30*24*60*60):

                            clean_memes_id.append(item) # запоминаем его 

            except:   # если питон говорит, что ошибся, сделай вот это 
                pass  # (ничего не делай) 
    

    # можно выводить инфу на экран вот так
    print('остаось {} человек'.format(len(clean_memes_id)))
    
    return clean_memes_id # вернули из функции чистые id

In [33]:
%%time
clean_members = { } # сюда будем класть всех прочищеных членов групп по очереди

for k,v in members.items():
    print('Фильтрую id из группы', k)
    infa = filter_members(v)
    clean_members[k] = infa  

Фильтрую id из группы foto_forsoul
остаось 24506 человек
Фильтрую id из группы artmilart
остаось 22920 человек
Фильтрую id из группы phmaste
остаось 20921 человек
Фильтрую id из группы club_of_photoshop
остаось 21796 человек
Фильтрую id из группы lr_school
остаось 26824 человек
CPU times: user 24.6 s, sys: 2.02 s, total: 26.6 s
Wall time: 9min 44s


In [34]:
with open('group_clean_members_extended.pickle', 'wb') as f:
    pickle.dump(clean_members, f)

In [48]:
import pickle
with open('group_clean_members_extended.pickle', 'rb') as f:
    clean_members = pickle.load(f)

In [63]:
clean_members['foto_forsoul'][0]

{'id': 3100364,
 'first_name': 'Юлия',
 'last_name': 'Плотникова',
 'is_closed': False,
 'can_access_closed': True,
 'sex': 1,
 'bdate': '14.8',
 'city': {'id': 10, 'title': 'Волгоград'},
 'last_seen': {'time': 1576774092, 'platform': 4},
 'home_town': 'Волгоград'}

In [41]:
def get_wall(user_id):
    # скачели стену 
    wall = vk_download('wall.get', 'owner_id='+str(user_id)+'&count=100')
    
    # распаковали её
    try:
        wall = wall['response']['items']
        post_texts = [cur['text'] for cur in wall]
        repost_texts = [cur['copy_history'][0]['text'] if 'copy_history' in cur else '' for cur in wall]

        # соединили все посты и репосты в один длинный текст
        wall_text = ' '.join(post_texts) + ' '.join(repost_texts)
        return wall_text
    except:
        return ''

In [115]:
import numpy as np

#Из-за ограничения запросов в вк по одному аккаунту, будем качать каждую группу отдельно, с разных токенов

for k,v in list(clean_members.items())[3:4]:
    print('Качаю стены участников из группы', k)    
    wall_members = [ ] # сюда будем сохранять стенки
    
    cur_members = np.array(v) 
    cur_sample = np.random.choice(cur_members, 1000, replace = False)
    
    for item in cur_sample:
    
        time.sleep(0.4) # отдыхаем между запросами 
        
        # попробуй, если код упал, напечатай мне id пользователя
        try:
            # распаковываем стену пользователя 
            wall_text = get_wall(item['id'])

            cur_item = {'group': k, 'id': item['id'], 'wall': wall_text}

            wall_members.append(cur_item)
        except:
            print('Ошибка!', item) 

    # сохраняем итоговые тексты стенок для текущего сообщества 
    with open(k + '.pickle', 'wb') as f:
        pickle.dump(wall_members, f)


Качаю стены участников из группы club_of_photoshop
